In [ ]:
#####################################################################################
# 카카오맵 API
# - 카카오맵에서 검색한 결과의 위도, 경도, 주소 등을 csv로 저장한다.
#####################################################################################
import pandas as pd
import requests
import pprint
import re

# kakao API key
MY_API_KEY = ""
API_REQUEST_URL = "https://dapi.kakao.com/v2/local/search/keyword.json?query={}"
API_REQUEST_HEADERS = {"Authorization": MY_API_KEY}
# csv 파일 경로
FILE_PATH = "관광지분류.csv"

result_list = []    # 카카오맵 위도경도 관광지
no_result_list = [] # 카카오맵 위도경도 데이터 없는 관광지

regexp = "[\(\)\[\]\<\>`\']" # 장소명에 괄호있는 부분 빼고 검색하기 위해 사용(카카오맵에서 검색안됨)

csv_data = pd.read_csv(FILE_PATH, names=["분류","장소명"], encoding="utf-8")

for i in range(0, len(csv_data)):
    location = csv_data.iloc[i]
    category_name = location["분류"]
    location_name_o = re.split(regexp, location["장소명"].replace(" ", ""))[0] # whitespace랑 괄호 안에 있는 문자 제외
                                                             
    url_by_location = API_REQUEST_URL.format(location_name_o)
    response = requests.get(url_by_location, headers=API_REQUEST_HEADERS)
    data = response.json()
    # pprint.pprint(data)
    
    result = data["documents"]
    
    if len(result) > 0:
        result_0 = result[0]
        latitude = result_0["y"]  # 위도
        longitude = result_0["x"] # 경도
        location_name_m = result_0["place_name"]    # 카카오맵상 장소명
        location_address = result_0["address_name"] # 주소
        print(i, ":", url_by_location)
        
        result_list.append((category_name, location_name_o, location_name_m, latitude, longitude, location_address))
        
    else:
        no_result_list.append((category_name, location_name_o))
        print("*위치없음->", i, ":", location_name_o)
    
result_df = pd.DataFrame(result_list, columns=["분류", "장소명", "카카오맵_장소명", "위도", "경도", "주소"])
no_result_df = pd.DataFrame(no_result_list, columns=["분류","장소명"])

result_df.to_csv("kakaomap_result.csv", 
                 mode="w", header=True, index=False, encoding="euc-kr")
no_result_df.to_csv("kakaomap_no_result.csv",
                    mode="w", header=True, index=False, encoding="euc-kr")

print("총 데이터 :", len(csv_data))
print("위치있음 :", len(result_df))
print("위치없음 :", len(no_result_df))
print("끝")